<a href="https://colab.research.google.com/github/scskalicky/LING-226-vuw/blob/main/26_Emotional_Vocab_in_The_Current.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Emotional Vocabulary in The Current**

In this notebook, we will play around with analysing the emotional vocabulary in The Current. Recall that in this data, people provided their responses to the question using a scale like this:
```
0: Excited
1: Hopeful
2: Don't care
3: Worried
4: Angry
```
Can we use a lexicon of emotion words to find patterns among answers in these different categories? Will answers made for "angry" include more angry words when compared to answers for "excited"? We can answer this question using a lexicon of emotion words and the text data from The Current.


The first thing we need to do is read in some data. Let's read in the data for two questions: whether petrol cars should be banned, and whether gene editing instead of poison should be used to control wasps. Unlike prior analyses with this data, we will retain the numerical score people provided for their answers.



In [ ]:
# petrol cars should be banned by 2050

# load the TP001 data to the notebook environment
!wget 'https://raw.githubusercontent.com/scskalicky/LING-226-vuw/main/the-current/tp001.txt'

In [ ]:
# open the text file and split on newlines
# rstrip is to clean the final newline that exists in each file.
tp001 = open('tp001.txt').read().rstrip().split('\n')

In [ ]:
# peek at the data
tp001[10:13]

### **Creating `(rating, comment)` tuples.**

Let's retain the comments and their numbers, we can create a set of tuples in the form of (rating, comment). We will also conver the number to `int()`. We can do all of this in a list comprehension by splitting on `'\t'` character which separates the rating and the comment. Note that `int()` is also in there to convert the rating to a number, although that's not really necessary for what we do in this analysis.

In [ ]:
tp001_data = [(int(answer.split('\t')[0]), answer.split('\t')[1]) for answer in tp001]

In [ ]:
# peek at the data
tp001_data[10:13]

How can we compare among the different ratings? There are only five total categories, so one method would be to create a dictionary, where they keys are the ratings, and the values are the different comments with those ratings.

We can use a `defaultdict` and a `for loop` to create this dictionary. First, create a `defaultdict` with a `list` as default. Then, loop through each of the (rating, comment) tuples. Slice the first item in the tuple and input it as the key to the dictionary - the keys will thus be 0, 1, 2, 3, or 4, reflecting the different ratings. Then, we can combinbe the value of that key to be whatever is currently in the dictionary for that key plus whatever the comment text is. If we wrap the comment text in a list, we can do this with list concatenation.



In [ ]:
from collections import defaultdict
banPetrol_dict = defaultdict(list)

for answer in tp001_data:
  banPetrol_dict[answer[0]] = banPetrol_dict[answer[0]] + [answer[1]]

In [ ]:
# 0 = excited
banPetrol_dict[0][14:20]

In [ ]:
# 4 = angry
banPetrol_dict[4][14:20]

Do the same thing for the other question. Read in the data, convert it to `(rating, comment)` tuples, and then use `defaultdict()` to create a dictionary where they keys are the rating `0, 1, 2, 3, or 4` and the values are a list of all the comments with that rating.





In [ ]:
# gene editing should be used to control wasps instead of poison
# load the TP012 data to the notebook environment
!wget 'https://raw.githubusercontent.com/scskalicky/LING-226-vuw/main/the-current/tp012.txt'

In [ ]:
# read in the entire file
tp012 = open('tp012.txt').read().rstrip().split('\n')
tp012_data = [(int(answer.split('\t')[0]), answer.split('\t')[1]) for answer in tp012]

In [ ]:
geneWasp_dict = defaultdict(list)

for answer in tp012_data:
  geneWasp_dict[answer[0]] = geneWasp_dict[answer[0]] + [answer[1]]

In [ ]:
# 0 = excited
geneWasp_dict[1][4:7]

In [ ]:
# 4 = angry
geneWasp_dict[4][4:7]

## **loading in emotional vocabulary**

Now that the question data is loaded, we can load in the lexicon of emotion words from the General Inquirer list. Load in the data by pulling it from the GitHub URL:

In [ ]:
# GI list
import requests

gi_url = 'https://raw.githubusercontent.com/scskalicky/LING-226-vuw/main/lexical-resources/inquirerbasic.txt'
raw_gi = requests.get(gi_url).text.split('\n')

# we need to do something a bit different for this resource
gi_dict = dict()

for category in raw_gi:
  gi_dict[category.split('\t')[0]] = category.split('\t')[1:]

Inspecting the keys shows us how many different categories are in this list:

In [ ]:
gi_dict.keys()

Which categories might work best for this analysis? Well, the `.._NRC` categories are a bit newer, so lets use the `Anger_NRC`, `Fear_NRC`, `Joy_NRC` to capture related emotions, as well as the `Positive_NRC` and `Negative_NRC` categories. We can save these values as a list, which can be used to loop through these emotions.

In [ ]:
emotion_list = ['Anger_NRC', 'Fear_NRC', 'Joy_NRC', 'Positive_NRC', 'Negative_NRC']

We will need to tokenize, so get that resource loaded in as well.

In [ ]:
# import nltk tokenizer functions
import nltk
nltk.download("punkt")

The goal is to see how many of the words in the answers from The Current data fall into each of these categories. To do so, we need to tokenize the answer data, and then capture just the tokens that fall into each one of these categories.

For example, if we wanted to grab all of the words associated with a `4` (Angry) that are also in the `Angry_NRC` category, we can run a one-liner list comprehension.

In this list comprehension, the answers associated with the rating of `4` are joined into a single string using `' '.join()`. That string is tokenized, and the lowercased version of each token is checked whether it exists in the `Anger_NRC` category of the lexicon. The result is a list of all the Angry words associated with responses of `4` in the question about banning petrol cars.

Inspect the output, which contains some offensive words, fair warning. But what do you think? Are these words angry?

In [ ]:
[token for token in nltk.word_tokenize(' '.join(banPetrol_dict[4])) if token.lower() in gi_dict["Anger_NRC"]]

Using this list comprehension, we can develop a function to extract all of the words associated with any comment / answer pairing. Below I create `analyse_emotions`, which will allow us to query any such pair we like:


In [ ]:
def analyse_emotions(text, emotion):
  """
  Args:
    text: a string
    emotion: name of key in gi_dict
  Returns:
    a list of tokens
  """
  tokens = nltk.word_tokenize(text)
  emotion_words = [word for word in tokens if word.lower() in gi_dict[emotion]]
  return emotion_words

Test the function:

In [ ]:
analyse_emotions(text = ' '.join(banPetrol_dict[4]), emotion = 'Anger_NRC')

We can now loop through the emotions and data as we like to obtain different lists of emotion words.

In [ ]:
# Look at the words associated with Angry answers in the question about banning petrol cars.
for emotion in emotion_list:
  emotion_words = analyse_emotions(text = ' '.join(banPetrol_dict[4]), emotion = emotion)
  print(f"{emotion}: {emotion_words}")

And, we can complexify the loop to look at each of the different answer categories and also count the number of words in each category using `len()`.

I also modified the answer list so that it contains the human readable label. I slice the first value in the loop, but print the second value so that is shows the label instead of the number. You can confirm [The Current metatdata scheme here.](https://catalogue.data.govt.nz/dataset/72ee59f3-cafb-4e3e-b8b7-01094c616216/resource/75fabd00-fdbf-40bd-bed7-ac769d5f73d2/download/general-read-me-for-the-current.txt)


Inspect the output - is there a greater number of angry words for answers in the `Angry` category? What about answers in the `Excited` category? What might be going on here? What about other comparisons, such as between positive and negative?


In [ ]:
for answer in [(0, 'Excited'),(1, "Hopeful"),(2, "Don't Care"),(3, "Worried"),(4, "Angry")]:
  for emotion in emotion_list:
      emotion_words = analyse_emotions(text = ' '.join(banPetrol_dict[answer[0]]), emotion = emotion)
      print(f"{answer[1]} has {len(emotion_words)} for {emotion}.The words are\n {emotion_words}")

Compare to the Wasp Data.

In [ ]:
for answer in [(0, 'Excited'),(1, "Hopeful"),(2, "Don't Care"),(3, "Worried"),(4, "Angry")]:
  for emotion in emotion_list:
      emotion_words = analyse_emotions(text = ' '.join(geneWasp_dict[answer[0]]), emotion = emotion)
      print(f"{answer[1]} has {len(emotion_words)} for {emotion}.The words are\n {emotion_words}")

## **Your Turn**

The approach above is interesting, but also suffers from a few issues which could be improved. For example, counting the length without normalising based on the total number of words is misleading. One way to improve upon this is to count the number of words in each category using `len()`, and then dividing that by the total number of words in each comment. This way you would get percentage comparisons between categories and questions.

You could also look within the results and use `nltk.FreqDist()` to compare the most / least frequency words used by people in different emotional categories.

You might also want to perform some additional comparisons of the data. Please spend the rest of the time working through some additional ideas, or building more advanced functions which help you to compare among the questions and categories.

